<a href="https://colab.research.google.com/github/puneetsingla17/Reinforcement-Learning-Pytorch/blob/master/PACMAN_env_dynamics_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install pybullet --upgrade
!pip install ptan
import pybullet_envs
import ptan
import math
import gym
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as op

Requirement already up-to-date: pybullet in /usr/local/lib/python3.6/dist-packages (2.8.4)


In [70]:
import cv2
from collections import deque,namedtuple
import random
from ptan.common.wrappers import ImageToPyTorch,ScaledFloatFrame,MaxAndSkipEnv,FrameStack,LazyFrames
from gym import spaces
import numpy as np

In [38]:
env=gym.make("MsPacmanNoFrameskip-v0")

In [4]:
#Description regarding environment space and action space
print(env.action_space)
print(env.observation_space)

Discrete(9)
Box(210, 160, 3)


In [5]:
#action means
env.unwrapped.get_action_meanings()

['NOOP',
 'UP',
 'RIGHT',
 'LEFT',
 'DOWN',
 'UPRIGHT',
 'UPLEFT',
 'DOWNRIGHT',
 'DOWNLEFT']

In [6]:
#Hyperparameters Dictionary
hyper={"maxskip":3,
       "nstack":4,
       }

In [7]:
class scaleframe(gym.ObservationWrapper):
  def __init__(self,env):
    super().__init__(env)
    self.env=env
    
  def observation(self,obs):
    return np.array(obs).astype(np.float32)/255.0

In [8]:
class ProcessFrame84(gym.ObservationWrapper):
  def __init__(self,env):
    super().__init__(env)
    self.env=env
    
  def observation(self,x):
    return ProcessFrame84.process(x)

  @staticmethod
  def process(ob):
    img=np.reshape(ob,[210,160,3]).astype(np.float32)
    img=img[:,:,0]*0.299+img[:,:,1]*0.587+img[:,:,2]*0.114
    resizedimg=cv2.resize(img,(84,110),interpolation=cv2.INTER_AREA)
    xt=resizedimg[18:102,:]
    xt=np.reshape(xt,(84,84,1))
    return xt.astype(np.uint8)

In [86]:
class FrameStack(gym.Wrapper):
    def __init__(self, env, k):
        """Stack k last frames.
        Returns lazy array, which is much more memory efficient.
        See Also
        --------
        baselines.common.atari_wrappers.LazyFrames
        """
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = (1,84,84)
        self.observation_space = spaces.Box(low=0, high=255, shape=(shp[0]*k, shp[1], shp[2]), dtype=np.float32)
        print(self.observation_space)

    def reset(self):
        ob = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob.reshape(1,84,84))
        return np.concatenate(self.frames,axis=0)

    def step(self, action):
        ob, reward, done, info = self.env.step(action)
        self.frames.append(ob.reshape(1,84,84))
        return np.concatenate(self.frames,axis=0), reward, done, info


In [87]:
# Preprocessing the environment 
# Normalizing scale of env values
# Concatenate multiple frames together to get environment dynamics
# Applying action for k consecutive frames and getting total reward out with obs as max of last two frames > maxandskipenv
# resizing to 84 84 1
env=gym.make("MsPacmanNoFrameskip-v0")
env=MaxAndSkipEnv(env,skip=hyper['maxskip'])
env=ProcessFrame84(env)
env=FrameStack(env,hyper['nstack'])
env=scaleframe(env)
env=ImageToPyTorch(env)

Box(4, 84, 84)


In [88]:
env.reset().shape

(84, 84, 4)

In [90]:
env.step(1)[0].shape

(84, 84, 4)